<a href="https://colab.research.google.com/github/morganFitzg/fashion_annotation/blob/main/Copy_of_first_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [62]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Apr  9 16:25:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    36W / 250W |    375MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence, to_categorical
from keras.losses import CategoricalCrossentropy

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import io 
from PIL import Image

# Setup

In [3]:
#config
IMAGE_SIZE=(224,224,3)
epochs=3
batch_size=32
path='/content/drive/My Drive/fashion_annotation/images/'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data=pd.read_csv('/content/drive/My Drive/fashion_annotation/data_v1.csv')
labels=data['str_label'].apply(eval)
data['category']=labels.apply(lambda x: x[0])
data['pattern']=labels.apply(lambda x: x[1])
#remove argyle and herringbone...there aren't enough for the model split
data=data[data['pattern']!='Argyle']
data=data[data['pattern']!='Herringbone']
cat_num=data['category'].nunique()
pat_num=data['pattern'].nunique()
print(cat_num)
print(pat_num)
data.drop(columns=['Unnamed: 0','str_label'],inplace=True)
#drop 's' from https in url (download only works if you change to http://)
data['url']=data['url'].apply(lambda x: x[:4]+x[5:])
data.head()

64
23


,imageId,num_label,url,category,pattern
0,1,"(95, 70)",http://contestimg.wish.com/api/webimage/570f35...,Jumpsuits Overalls & Rompers,Floral
1,2,"(36, 229)",http://contestimg.wish.com/api/webimage/5468f1...,Casual Dresses,None
2,3,"(170, 229)",http://contestimg.wish.com/api/webimage/546410...,Skirts,None
3,4,"(18, 229)",http://contestimg.wish.com/api/webimage/550b95...,Blouses,None
4,5,"(189, 181)",http://contestimg.wish.com/api/webimage/54451f...,Swimsuits,Stripes


In [6]:
#create numerical codes for the category and pattern labels
data.category=pd.Categorical(data.category)
data['cat_codes']=data.category.cat.codes
data.pattern=pd.Categorical(data.pattern)
data['pat_codes']=data.pattern.cat.codes

#make key for the codes
pattern_key=data.groupby('pattern').mean()['pat_codes'].astype(int)
category_key=data.groupby('category').mean()['cat_codes'].astype(int)
category_key

category
Athletic Pants         0
Athletic Shirts        1
Athletic Shorts        2
Batwing Tops           3
Blouses                4
                      ..
Underwear             59
Uniforms              60
Vests                 61
Wedding Dresses       62
Wedges & Platforms    63
Name: cat_codes, Length: 64, dtype: int64

In [7]:
#urls already identified as not working

bad_urls=['http://contestimg.wish.com/api/webimage/56bff1b758d90e15ce6678ab-large',
 'http://contestimg.wish.com/api/webimage/59266d2c7840080d92f71aae-large',
 'http://contestimg.wish.com/api/webimage/53b81620369fb20f1bdac5b1-large',
 'http://contestimg.wish.com/api/webimage/573d7d504c8aee5d2d442273-large',
 'http://contestimg.wish.com/api/webimage/56e62ec3a10d0f5c8bb2c76a-large',
 'http://contestimg.wish.com/api/webimage/53aecca613e1b37ec4a8a05f-large',
 'http://contestimg.wish.com/api/webimage/565d4726e389f7736f0c3273-large',
 'http://contestimg.wish.com/api/webimage/5967b80fddebc172b7621eef-large',
 'http://contestimg.wish.com/api/webimage/565930f14e364424964a299b-large',
 'http://contestimg.wish.com/api/webimage/58b1080f0d3459553b600110-large',
 'http://contestimg.wish.com/api/webimage/5212128d3deaf71357104a5d-large',
 'http://contestimg.wish.com/api/webimage/52f01f1f9f5ef7425bcb89b2-large',
 'http://contestimg.wish.com/api/webimage/53c43a6cbc06ac7c0383d50d-large',
 'http://contestimg.wish.com/api/webimage/53c3452e60beeb110c66254d-large',
 'http://contestimg.wish.com/api/webimage/522f851f3deaf70907580f2b-large',
 'http://contestimg.wish.com/api/webimage/565bbe67675dec12702cf2d3-large',
 'http://contestimg.wish.com/api/webimage/5901abf9513b402b44cc26b1-large',
 'http://contestimg.wish.com/api/webimage/57d29b5577ba9b048e030d5c-large',
 'http://contestimg.wish.com/api/webimage/57e5b3088e13b0132a6bedd1-large',
 'http://contestimg.wish.com/api/webimage/54646efb1280fa054336c95a-large',
 'http://contestimg.wish.com/api/webimage/590c3d34e0485212255dbc06-large',
 'http://contestimg.wish.com/api/webimage/56e7029b96ce815543f50502-large']

In [8]:
#remove bad urls
print(data.shape)
for url in bad_urls:
  data=data[data['url']!=url]

print(data.shape)

(877421, 7)
(877399, 7)


In [18]:
#remove already trained data
urls=train_data['url'].to_list()
data=data[~data['url'].isin(urls)]
data.shape

(811599, 7)

In [19]:
X=data['url']
y=data[['cat_codes','pat_codes']]
X_use, X_extra, y_use, y_extra = train_test_split(X,y,
                                                  test_size=.9,
                                                  random_state=42,
                                                  stratify=y['cat_codes'])

X_train, X_val, y_train, y_val = train_test_split(X_use,y_use,
                                                  test_size=.25,
                                                  random_state=35,
                                                  stratify=y_use['cat_codes'])

In [20]:
train_data=pd.concat([X_train,y_train],axis=1)
val_data=pd.concat([X_val,y_val],axis=1)

# Download Images

In [9]:
#function that takes url, downloads image and saves it to google drive
def write_im_data(url):

  response=requests.get(url,stream=True)
  im=response.content
  
  path='/content/drive/My Drive/fashion_annotation/images/'

  with open (path+url[-30:]+'.bin','wb') as f:
    f.write(im)

In [10]:
#Test if the urls in train_data and val_data are present in directory

def find_missing_urls(urls):
  bad_urls=[] 

  for url in urls:
    try:
      with open(path+url[-30:]+'.bin','rb') as f:
        im=f.read()
    except Exception as e:
      bad_urls.append(url)
    
  return bad_urls

#test if images in directory open properly
def test_urls(urls):
  bad_urls=[] 

  for url in train_data['url']:
    try:
      with open(path+url[-30:]+'.bin','rb') as f:
        im=f.read()
        byte_im=io.BytesIO(im)
        #byte_im.seek(0)
        img=Image.open(byte_im)
    except Exception as e:
      bad_urls.append(url)
  return bad_urls

In [ ]:
missing_urls=find_missing_urls(train_data)
val_missing=find_missing_urls(val_data)
missing_urls=missing_urls.extend(val_missing)

In [77]:
count=0
for url in missing_urls:
  write_im_data(url)
  count+=1
  print(count)


Streaming output truncated to the last 5000 lines.
46691
46692
46693
46694
46695
46696
46697
46698
46699
46700
46701
46702
46703
46704
46705
46706
46707
46708
46709
46710
46711
46712
46713
46714
46715
46716
46717
46718
46719
46720
46721
46722
46723
46724
46725
46726
46727
46728
46729
46730
46731
46732
46733
46734
46735
46736
46737
46738
46739
46740
46741
46742
46743
46744
46745
46746
46747
46748
46749
46750
46751
46752
46753
46754
46755
46756
46757
46758
46759
46760
46761
46762
46763
46764
46765
46766
46767
46768
46769
46770
46771
46772
46773
46774
46775
46776
46777
46778
46779
46780
46781
46782
46783
46784
46785
46786
46787
46788
46789
46790
46791
46792
46793
46794
46795
46796
46797
46798
46799
46800
46801
46802
46803
46804
46805
46806
46807
46808
46809
46810
46811
46812
46813
46814
46815
46816
46817
46818
46819
46820
46821
46822
46823
46824
46825
46826
46827
46828
46829
46830
46831
46832
46833
46834
46835
46836
46837
46838
46839
46840
46841
46842
46843
46844
46845
46846
46847
46848
4

In [80]:
print(train_data.shape)
print(val_data.shape)
for url in bad_urls:
  train_data=train_data[train_data['url']!=url]
  val_data=val_data[val_data['url']!=url]

print(train_data.shape)
print(val_data.shape)

(65805, 3)
(21935, 3)
(65800, 3)
(21933, 3)


In [21]:
train_data.to_csv('/content/drive/My Drive/fashion_annotation/train_data_1.csv',index=False)
val_data.to_csv('/content/drive/My Drive/fashion_annotation/val_data_1.csv',index=False)

# Load Prepared Data

In [11]:
train_data=pd.read_csv('/content/drive/My Drive/fashion_annotation/train_data.csv')
val_data=pd.read_csv('/content/drive/My Drive/fashion_annotation/val_data.csv')

In [ ]:
cat_num=len(train_data['cat_codes'].unique())
pat_num=len(train_data['pat_codes'].unique())
print(cat_num)
print(pat_num)

64
25


In [ ]:
len(val_data['cat_codes'].unique())

64

In [ ]:
len(val_data['pat_codes'].unique())

24

In [ ]:
y_col_dict={'pattern':'pat_codes','category':'cat_codes'}

# Build Custom Image Generator

Build a custom data generator that downloads images for a batch, resize, convert to numpy array.  We need a custom generator since we are downloading the files directly from a url and converting from byte encoded format.

Look at image augmentation options once model is working.

In [82]:
class CustomImageGen(Sequence):
  def __init__(self, df, X_col, y_col,
               batch_size,
               input_size=(224,224,3),
               shuffle=True):
    self.df = df
    self.X_col = X_col
    self.y_col = y_col
    self.batch_size = batch_size
    self.input_size = input_size
    self.shuffle = shuffle 

    self.n = len(self.df)
    self.n_cat = df[y_col['category']].nunique()
    self.n_pat = df[y_col['pattern']].nunique()
  
  def on_epoch_end(self):
    if self.shuffle:
      self.df = self.df.sample(frac=1).reset_index(drop=True)

  def __getim_(self,url):
    """
    download image from url, open from byte string format,
    resize and convert to np array (224,224,3)
    """
    #print(url)
    #response=requests.get(url,stream=True)
    #im=response.content
    with open(path+url[-30:]+'.bin','rb') as f:
      im=f.read()
    byte_im=io.BytesIO(im)
    #byte_im.seek(0)
    img=Image.open(byte_im)
    img_sized=img.resize((224,224))
    im_array=np.asarray(img_sized)
    return im_array

  def __get_images(self,url_batch):
    """
    initiate image array for batch, 
    call __getim_ to load each image to array
    preprocess batch for VGG and return array size (batch_size,224,224,3)
    """
    dims=[x for x in self.input_size]
    im_array=np.zeros((self.batch_size,dims[0],dims[1],dims[2]))

    for count,url in enumerate(url_batch):
      im_array[count,:,:,:]=self.__getim_(url)

    #preprocess for VGG and return
    return preprocess_input(im_array)

  def __get_data(self, batches):
    url_batch = batches[self.X_col]

    cat_batch = batches[self.y_col['category']]
    pat_batch = batches[self.y_col['pattern']]

    X_array = self.__get_images(url_batch)

    #ohe the output labels
    y_cat=to_categorical(cat_batch,self.n_cat)
    y_pat=to_categorical(pat_batch,self.n_pat)

    return X_array, (y_cat,y_pat)

  def __getitem__(self, index):
    batches=self.df[index*self.batch_size:(index+1)*self.batch_size]
    X, y = self.__get_data(batches)
    return X, y

  def __len__(self):
    return self.n // self.batch_size


# Build Model

In [83]:
vgg=VGG16(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
vgg.trainable=False

58900480/58889256 [==============================] - 1s 0us/step


In [84]:
inputs=Input(shape=IMAGE_SIZE)

x = vgg(inputs, training=False)

#Convert features of output shape to vectors
branch = GlobalAveragePooling2D()(x)

#Category Prediction Branch
cat_branch1 = Dense(512, activation='relu')(branch)
cat_branch2 = Dense(256, activation='relu')(cat_branch1)
cat_pred=Dense(cat_num, activation='softmax')(cat_branch2)

#Pattern Prediction Branch

pat_branch1 = Dense(512, activation='relu')(branch)
pat_branch2 = Dense(256, activation='relu')(pat_branch1)
pat_pred=Dense(pat_num, activation='softmax')(pat_branch2)

model=Model(inputs,outputs=[cat_pred,pat_pred])

In [85]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 vgg16 (Functional)             (None, 7, 7, 512)    14714688    ['input_2[0][0]']                
                                                                                                  
 global_average_pooling2d (Glob  (None, 512)         0           ['vgg16[0][0]']                  
 alAveragePooling2D)                                                                              
                                                                                              

In [86]:
# Defining losses for each output
losses ={ 'dense_2':CategoricalCrossentropy(),
    'dense_5':CategoricalCrossentropy()}

model.compile(
    loss=losses,
    optimizer='adam',
    metrics=['accuracy']
)

In [87]:
traingen = CustomImageGen(
    train_data, 'url', y_col_dict, batch_size=32
)

valgen = CustomImageGen(
    val_data, 'url', y_col_dict, batch_size=32
)


results = model.fit(traingen,
                    validation_data=valgen,
                    epochs=3)

Epoch 1/3
2056/2056 [==============================] - 1091s 526ms/step - loss: 3.3241 - dense_2_loss: 2.2745 - dense_5_loss: 1.0496 - dense_2_accuracy: 0.3913 - dense_5_accuracy: 0.7449 - val_loss: 2.9895 - val_dense_2_loss: 2.0256 - val_dense_5_loss: 0.9639 - val_dense_2_accuracy: 0.4269 - val_dense_5_accuracy: 0.7480
Epoch 2/3
2056/2056 [==============================] - 1160s 564ms/step - loss: 2.8508 - dense_2_loss: 1.9300 - dense_5_loss: 0.9208 - dense_2_accuracy: 0.4508 - dense_5_accuracy: 0.7557 - val_loss: 2.8817 - val_dense_2_loss: 1.9486 - val_dense_5_loss: 0.9331 - val_dense_2_accuracy: 0.4505 - val_dense_5_accuracy: 0.7548
Epoch 3/3
2056/2056 [==============================] - 1157s 563ms/step - loss: 2.6845 - dense_2_loss: 1.8070 - dense_5_loss: 0.8775 - dense_2_accuracy: 0.4746 - dense_5_accuracy: 0.7602 - val_loss: 2.9139 - val_dense_2_loss: 1.9697 - val_dense_5_loss: 0.9442 - val_dense_2_accuracy: 0.4452 - val_dense_5_accuracy: 0.7537


In [88]:
import pickle
with open('/content/drive/My Drive/fashion_annotation/model_1.pickle', 'wb') as f:
  pickle.dump(model, f)

with open('/content/drive/My Drive/fashion_annotation/results_mod1.pickle', 'wb') as f:
  pickle.dump(results, f)

INFO:tensorflow:Assets written to: ram://66299161-bd2e-4e2c-a780-3611319b24d1/assets
INFO:tensorflow:Assets written to: ram://f8e1d0ba-a274-4dcf-9d1a-eba14278e554/assets


In [ ]:
urls=train_data['url']
bad_urls=[] 

for url in urls:
  try:
    with open(path+url[-30:]+'.bin','rb') as f:
      im=f.read()
    byte_im=io.BytesIO(im)
    #byte_im.seek(0)
    img=Image.open(byte_im)
  except Exception as e:
    bad_urls.append(url)

bad_urls

['http://contestimg.wish.com/api/webimage/56bff1b758d90e15ce6678ab-large',
 'http://contestimg.wish.com/api/webimage/59266d2c7840080d92f71aae-large',
 'http://contestimg.wish.com/api/webimage/53b81620369fb20f1bdac5b1-large',
 'http://contestimg.wish.com/api/webimage/573d7d504c8aee5d2d442273-large',
 'http://contestimg.wish.com/api/webimage/56e62ec3a10d0f5c8bb2c76a-large',
 'http://contestimg.wish.com/api/webimage/53aecca613e1b37ec4a8a05f-large',
 'http://contestimg.wish.com/api/webimage/565d4726e389f7736f0c3273-large',
 'http://contestimg.wish.com/api/webimage/5967b80fddebc172b7621eef-large',
 'http://contestimg.wish.com/api/webimage/565930f14e364424964a299b-large',
 'http://contestimg.wish.com/api/webimage/58b1080f0d3459553b600110-large',
 'http://contestimg.wish.com/api/webimage/5212128d3deaf71357104a5d-large']

In [ ]:
bad_urls=['http://contestimg.wish.com/api/webimage/56bff1b758d90e15ce6678ab-large',
 'http://contestimg.wish.com/api/webimage/59266d2c7840080d92f71aae-large',
 'http://contestimg.wish.com/api/webimage/53b81620369fb20f1bdac5b1-large',
 'http://contestimg.wish.com/api/webimage/573d7d504c8aee5d2d442273-large',
 'http://contestimg.wish.com/api/webimage/56e62ec3a10d0f5c8bb2c76a-large',
 'http://contestimg.wish.com/api/webimage/53aecca613e1b37ec4a8a05f-large',
 'http://contestimg.wish.com/api/webimage/565d4726e389f7736f0c3273-large',
 'http://contestimg.wish.com/api/webimage/5967b80fddebc172b7621eef-large',
 'http://contestimg.wish.com/api/webimage/565930f14e364424964a299b-large',
 'http://contestimg.wish.com/api/webimage/58b1080f0d3459553b600110-large',
 'http://contestimg.wish.com/api/webimage/5212128d3deaf71357104a5d-large']

In [ ]:
train_data.shape

(65814, 3)

In [ ]:
val_data.shape

(21938, 3)

In [ ]:
gen = CustomImageGen(
    train_data, 'url', y_col_dict, batch_size=32
)

batch=gen.__getitem__(0)

In [ ]:
batch

(array([[[[  43.061,   15.221,   59.32 ],
          [  44.061,   16.221,   60.32 ],
          [  45.061,   17.221,   61.32 ],
          ...,
          [  43.061,   17.221,   62.32 ],
          [  42.061,   17.221,   60.32 ],
          [  42.061,   17.221,   59.32 ]],
 
         [[  44.061,   16.221,   60.32 ],
          [  44.061,   16.221,   60.32 ],
          [  45.061,   17.221,   61.32 ],
          ...,
          [  45.061,   19.221,   63.32 ],
          [  43.061,   18.221,   61.32 ],
          [  41.061,   16.221,   57.32 ]],
 
         [[  44.061,   16.221,   60.32 ],
          [  45.061,   17.221,   61.32 ],
          [  46.061,   18.221,   62.32 ],
          ...,
          [  45.061,   19.221,   63.32 ],
          [  43.061,   18.221,   61.32 ],
          [  42.061,   17.221,   58.32 ]],
 
         ...,
 
         [[ -12.939,  -28.779,   33.32 ],
          [ -12.939,  -28.779,   33.32 ],
          [ -11.939,  -27.779,   34.32 ],
          ...,
          [ -66.939,  -46.779,  -